In [ ]:
import sys
sys.path.insert(0, '..')

import os.path as op

import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

import numpy as np

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table

import pickle

In [ ]:
# PAUS catalog
sel_dfs = []
region_list = ['W1', 'W2', 'W3']
LFs_dir = '/home/alberto/almacen/PAUS_data/Lya_LFs'
nb_list = [[0, 2], [2, 4], [4, 6], [6, 8],
           [8, 10], [10, 12], [12, 14], [14, 16],
           [16, 18]]
for [nb1, nb2] in nb_list:
    for jj, region in enumerate(region_list):
        with open(f'{LFs_dir}/Lya_LF_nb{nb1}-{nb2}_{region}/selection.pkl', 'rb') as file:
            this_dict = pickle.load(file)
            # Separate the L_lya err into two columns
            this_dict['L_lya_corr_err_up'] = this_dict['L_lya_corr_err'][1]
            this_dict['L_lya_corr_err_down'] = this_dict['L_lya_corr_err'][0]
            this_dict['ref_id'] = this_dict['ref_id']
            del this_dict['L_lya_corr_err']
        
        sel_dfs.append(pd.DataFrame(this_dict))
selection = pd.concat(sel_dfs)
selection['ref_id'] = selection['ref_id'].astype(int)
selection_0 = selection.copy()
selection = selection.drop_duplicates(subset='ref_id')

nice_lya = np.asarray(selection['nice_lya'])
selection = selection[nice_lya]

selection = selection.reset_index(drop=True)
print('PAUS cat loaded.')

# SDSS catalog
path_to_cat = '/home/alberto/almacen/SDSS_spectra_fits/DR18/spAll-v5_13_2.fits'
sdss_cat = fits.open(path_to_cat)[1].data
print('SDSS cat loaded.')

# HETDEX catalog
path_to_cat = '/home/alberto/almacen/HETDEX_catalogs/hetdex_source_catalog_1'
version = 'v3.2'
hetdex_cat = pd.DataFrame(
    dict(Table.read(op.join(path_to_cat, f'hetdex_sc1_{version}.ecsv')))
    )
print('HETDEX cat loaded.')

# DESI catalog
path_to_cat = '/home/alberto/almacen/DESI/zall-pix-fuji.fits'
desi_cat = fits.open(path_to_cat)[1].data
print('DESI cat loaded.')

In [ ]:
print(f'{len(selection)=}')

fig, ax = plt.subplots(figsize=(6, 4))

ax.hist(selection['z_NB'], 100)

ax.set_ylabel('N')
ax.set_xlabel('Redshift')

plt.show()

In [ ]:
# Do the cross-matches
coords_paus = SkyCoord(ra=np.asarray(selection['RA']) * u.deg,
                       dec=np.asarray(selection['DEC']) * u.deg)
coords_sdss = SkyCoord(ra=sdss_cat['RA'] * u.deg,
                       dec=sdss_cat['DEC'] * u.deg)
coords_hetdex = SkyCoord(ra=np.asarray(hetdex_cat['RA']) * u.deg,
                         dec=np.asarray(hetdex_cat['DEC']) * u.deg)
coords_desi = SkyCoord(ra=desi_cat['TARGET_RA'] * u.deg,
                       dec=desi_cat['TARGET_DEC'] * u.deg)

xm_id_sdss, ang_dist_sdss, _= coords_paus.match_to_catalog_sky(coords_sdss)
xm_id_hetdex, ang_dist_hetdex, _= coords_paus.match_to_catalog_sky(coords_hetdex)
xm_id_desi, ang_dist_desi, _= coords_paus.match_to_catalog_sky(coords_desi)

# Objects with 1 arcsec of separation
mask_dist_sdss = (ang_dist_sdss <= 1.5 * u.arcsec) & (sdss_cat['ZWARNING'][xm_id_sdss] == 0)
mask_dist_desi = (ang_dist_desi <= 1.5 * u.arcsec) & (desi_cat['ZWARN'][xm_id_desi] == 0)
mask_dist_hetdex = (ang_dist_hetdex <= 1.5 * u.arcsec) & (hetdex_cat['z_hetdex_conf'][xm_id_hetdex] > 0.9)

mask_dist_sdss = np.asarray(mask_dist_sdss)
mask_dist_desi = np.asarray(mask_dist_desi)
mask_dist_hetdex = np.asarray(mask_dist_hetdex)

In [ ]:
from paus_utils import z_NB

fig, ax = plt.subplots(figsize=(6, 3.5))


ax.plot(sdss_cat['Z'][xm_id_sdss][mask_dist_sdss],
        selection['z_NB'][mask_dist_sdss],
        ls='', marker='o', ms=6, mew=1.5,
        mec='g', mfc='none', zorder=11)
ax.plot(np.asarray(hetdex_cat['z_hetdex'])[xm_id_hetdex][mask_dist_hetdex],
        selection['z_NB'][mask_dist_hetdex],
        ls='', marker='^', ms=6, mew=1.5,
        mec='peru', mfc='none', zorder=13)
ax.plot(np.asarray(desi_cat['Z'])[xm_id_desi][mask_dist_desi],
        selection['z_NB'][mask_dist_desi],
        ls='', marker='x', ms=6, mew=1.5,
        mec='deepskyblue', mfc='none', zorder=12)

# lines
zsp_xx = np.linspace(0, 5, 50)
w_lya = 1215.67
w_CIV = 1549.48
w_CIII = 1908.73
w_MgII = 2799.12
w_OII = 3727
w_OIII = (4960 + 5008) * 0.5
w_Hbeta = 4862
ax.plot(zsp_xx, zsp_xx, ls='-', c='r', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_CIV - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_CIII - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_MgII - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_OII - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_OIII - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)
ax.plot((zsp_xx + 1) * w_lya / w_Hbeta - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)

ax.plot((zsp_xx + 1) * w_lya / 1025 - 1, zsp_xx, 
        ls='--', c='dimgray', zorder=-99)

text_plot = [[0.3, 3.9, '[OII]'],
             [0.01, 4.4, '[OIII]'],
             [0.37, 4.3, r'H$\beta$'],
             [0.8, 3.9, 'MgII'],
             [1.8, 3.9, 'CIII]'],
             [2.6, 3.9, 'CIV'],
             [3.5, 3.9, r'\bf Ly$\mathbf{\alpha}$'],
             [4.0, 3.5, r'Ly$\beta$']]
for text in text_plot:
        ax.text(text[0], text[1], text[2], fontsize=11,
                zorder=99999)

ax.set_xlim(-0.1, 4.5)
ax.set_ylim(2.5, 4.5)

ax.set_ylabel(r'$z_\mathrm{NB}$', fontsize=15)
ax.set_xlabel(r'$z_\mathrm{spec}$', fontsize=15)
ax.tick_params(direction='in', which='both', labelsize=13)
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

plt.show(block=False)

fig, ax = plt.subplots()

ax.plot(selection['z_NB'][mask_dist_sdss],
        selection['z_NB'][mask_dist_sdss] - sdss_cat['Z'][xm_id_sdss][mask_dist_sdss],
        ls='', marker='o', ms=6, mew=1.5,
        mec='g', mfc='none', zorder=11, label='SDSS')
ax.plot(selection['z_NB'][mask_dist_hetdex],
        selection['z_NB'][mask_dist_hetdex] - np.asarray(hetdex_cat['z_hetdex'])[xm_id_hetdex][mask_dist_hetdex],
        ls='', marker='^', ms=6, mew=1.5,
        mec='peru', mfc='none', zorder=13, label='HETDEX')
ax.plot(selection['z_NB'][mask_dist_desi],
        selection['z_NB'][mask_dist_desi] - np.asarray(desi_cat['Z'])[xm_id_desi][mask_dist_desi],
        ls='', marker='x', ms=6, mew=1.5,
        mec='deepskyblue', mfc='none', zorder=12, label='DESI')

# NB redshifts
NB_redshift = z_NB(selection['lya_NB'])
ax.plot(NB_redshift[mask_dist_sdss],
        NB_redshift[mask_dist_sdss] - sdss_cat['Z'][xm_id_sdss][mask_dist_sdss],
        ls='', marker='.', ms=6,
        c='k', zorder=-99, alpha=0.4)
ax.plot(NB_redshift[mask_dist_hetdex],
        NB_redshift[mask_dist_hetdex] - np.asarray(hetdex_cat['z_hetdex'])[xm_id_hetdex][mask_dist_hetdex],
        ls='', marker='.', ms=6,
        c='k', zorder=-99, alpha=0.4)
ax.plot(NB_redshift[mask_dist_desi],
        NB_redshift[mask_dist_desi] - np.asarray(desi_cat['Z'])[xm_id_desi][mask_dist_desi],
        ls='', marker='.', ms=6,
        c='k', zorder=-99, alpha=0.4,
        label='NB redshift')

ax.errorbar(4.2, 0.08, yerr=0.053, mec='k',
            mfc='w', fmt='o', capsize=2, mew=1.5,
            ms=6, ecolor='k')
ax.text(4.125, 0.0775, 'NB width =', fontsize=13,
        horizontalalignment='right')

ax.axhline(0, c='k', ls='--', zorder=-9999)

ax.set_xlim(2.5, 4.3)
ax.set_ylim(-0.1, 0.15)

ax.set_xlabel(r'$z_{\rm phot}$')
ax.set_ylabel(r'$z_{\rm phot} - z_{\rm spec}$')

ax.legend(fontsize=11)

plt.show()

In [ ]:
# Define global z_spec
z_spec = np.ones_like(mask_dist_desi).astype(float) * -1
z_spec[mask_dist_hetdex] = hetdex_cat['z_hetdex'][xm_id_hetdex][mask_dist_hetdex]
z_spec[mask_dist_desi] = desi_cat['Z'][xm_id_desi][mask_dist_desi]
z_spec[mask_dist_sdss] = sdss_cat['Z'][xm_id_sdss][mask_dist_sdss]
selection['z_spec'] = z_spec

In [ ]:
save_sel_to = '/home/alberto/almacen/PAUS_data/catalogs/LAE_selection.csv'
selection_0.to_csv(save_sel_to)

In [ ]:
from jpasLAEs.utils import smooth_hist

fig, ax = plt.subplots(figsize=(6, 4))

mask_L = (selection['L_lya_corr'] > 44) & (selection['EW0_lya'] > 00)
sel_mask = (z_spec > 0) & mask_L

# Define nice z
nice_z = np.abs(selection['z_NB'] - z_spec) < 0.15

nice_mask = sel_mask & nice_z

nice_h_smooth, to_plot_c = smooth_hist(selection['z_NB'][nice_mask],
                                       2.7, 4.5, 0.05, 0.1)
sel_h_smooth, to_plot_c = smooth_hist(selection['z_NB'][sel_mask],
                                       2.7, 4.5, 0.05, 0.1)

sdss_p = nice_h_smooth / sel_h_smooth
p_err = ((nice_h_smooth ** 0.5 / sel_h_smooth) ** 2
         + (sdss_p * sel_h_smooth ** -0.5) ** 2) ** 0.5
ax.errorbar(to_plot_c, sdss_p, lw=2, label='L_lya$>$44, EW0$>$0')

ax.legend()

ax.set_ylim(0, 1.01)
ax.set_xlabel('Redshift')
ax.set_ylabel('Purity')

plt.show(block=False)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

mask = (np.abs(z_spec - selection['z_NB']) >= 0.12) & (z_spec > 0)
ax.scatter(selection['L_lya'][mask],
           selection['class_star'][mask],
           label='Bad z')
mask = (np.abs(z_spec - selection['z_NB']) < 0.12) & (z_spec > 0)
ax.scatter(selection['L_lya'][mask],
           selection['class_star'][mask],
           label='Good z')

ax.set_xlabel(r'$\log_{10}L_{{\rm Ly}\alpha}$')
ax.set_ylabel('class_star')
ax.set_ylim(0, 1)
           
ax.legend()

plt.show()


fig, ax = plt.subplots(figsize=(6, 4))

mask = (np.abs(z_spec - selection['z_NB']) >= 0.12) & (z_spec > 0)
ax.scatter(selection['r_mag'][mask],
           selection['class_star'][mask],
           label='Bad z')
mask = (np.abs(z_spec - selection['z_NB']) < 0.12) & (z_spec > 0)
ax.scatter(selection['r_mag'][mask],
           selection['class_star'][mask],
           label='Good z')

ax.set_xlabel(r'$r$')
ax.set_ylabel('class_star')
ax.set_ylim(0, 1)
           
ax.legend()

plt.show()


fig, ax = plt.subplots(figsize=(6, 4))

mask = (np.abs(z_spec - selection['z_NB']) >= 0.12) & (z_spec > 0)
ax.scatter((selection['z_NB'] - z_spec)[mask],
           selection['class_star'][mask],
           label='Bad z')
mask = (np.abs(z_spec - selection['z_NB']) < 0.12) & (z_spec > 0)
ax.scatter((selection['z_NB'] - z_spec)[mask],
           selection['class_star'][mask],
           label='Good z')

ax.set_xlabel(r'$r$')
ax.set_ylabel('class_star')
ax.set_ylim(0, 1)
           
ax.legend()

plt.show()